# Machine Learning Pipeline - Feature Selection

In this notebook, we pick up the transformed datasets that we saved in the previous notebook.

## Reproducibility: Setting the seed

With the aim to ensure reproducibility between runs of the same notebook, but also between the research and production environment, for each step that includes some element of randomness, it is extremely important that we **set the seed**.

In [1]:
# to handle datasets
import pandas as pd
import numpy as np

# for plotting
import matplotlib.pyplot as plt

# to build the models
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

# to visualise al the columns in the dataframe
pd.pandas.set_option('display.max_columns', None)

In [2]:
# load the train and test set with the engineered variables

# we built and saved these datasets in the previous notebook.
# If you haven't done so, go ahead and check the previous notebook
# to find out how to create these datasets

X_train = pd.read_csv('xtrain.csv')
X_test = pd.read_csv('xtest.csv')

X_train.head()

,longitude,latitude,housing_median_age,total_rooms,population,median_income,ocean_proximity,income_bracket,rooms_per_household,bedrooms_per_household,population_per_household,median_income_per_household
0,1.990148,-0.831603,-0.584061,-0.822229,-0.693618,-0.181254,-1.253415,-0.481839,0.604762,2.352336,0.083696,-0.022035
1,0.958314,-1.031433,-1.063452,1.993611,1.469553,1.157683,-0.300872,2.012706,0.604762,-0.112234,0.083696,-0.022035
2,0.928406,-0.729364,1.892792,-1.633859,1.258138,-1.305856,-1.253415,-0.481839,-0.925870,-0.112234,0.083696,-0.022035
3,-0.796302,1.403711,-1.862437,1.910722,1.215999,1.251134,-1.253415,2.012706,0.604762,-0.112234,0.083696,-0.022035
4,-1.284803,0.962225,1.892792,0.418302,0.146383,0.267953,1.604215,-0.481839,-0.065493,-0.112234,0.083696,-0.022035


In [3]:
# load the target (remember that the target is transformed with Yeo-Johnson Transformation)
y_train = pd.read_csv('ytrain.csv')
y_test = pd.read_csv('ytest.csv')

y_train.head()

,median_house_value
0,11.180246
1,12.385473
2,12.266697
3,12.401639
4,12.119970


### Feature Selection

Let's go ahead and select a subset of the most predictive features. There is an element of randomness in the Lasso regression, so remember to set the seed.

In [4]:
# We will do the model fitting and feature selection
# altogether in a few lines of code

# first, we specify the Lasso Regression model, and we
# select a suitable alpha (equivalent of penalty).
# The bigger the alpha the less features that will be selected.

# Then we use the selectFromModel object from sklearn, which
# will select automatically the features which coefficients are non-zero

# remember to set the seed, the random state in this function
sel_ = SelectFromModel(Lasso(alpha=0.001, random_state=0))

# train Lasso model and select features
sel_.fit(X_train, y_train)

SelectFromModel(estimator=Lasso(alpha=0.001, random_state=0))

In [5]:
sel_.get_support().sum()

12

In [6]:
# let's visualise those features that were selected.
# (selected features marked with True)

sel_.get_support()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [7]:
# let's print the number of total and selected features

# this is how we can make a list of the selected features
selected_feats = X_train.columns[(sel_.get_support())]

# let's print some stats
print('total features: {}'.format((X_train.shape[1])))
print('selected features: {}'.format(len(selected_feats)))
print('features with coefficients shrank to zero: {}'.format(
    np.sum(sel_.estimator_.coef_ == 0)))

total features: 12
selected features: 12
features with coefficients shrank to zero: 0


In [8]:
# print the selected features
selected_feats

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'population', 'median_income', 'ocean_proximity', 'income_bracket',
       'rooms_per_household', 'bedrooms_per_household',
       'population_per_household', 'median_income_per_household'],
      dtype='object')

In [9]:
pd.Series(selected_feats).to_csv('selected_features.csv', index=False)